# Climate Profiles 2

### Step 0: Set-Up
Import the [climakitae](https://github.com/cal-adapt/climakitae) library and other dependencies.

In [1]:
from typing import Tuple

import numpy as np
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm  # Progress bar

import climakitae as ck
from climakitae.explore.standard_year_profile import get_climate_profile, export_profile_to_csv, retrieve_profile_data, _filter_by_ssp
from climakitae.util.warming_levels import get_gwl_at_year
from climakitae.explore.typical_meteorological_year import TMY
from climakitae.core.data_interface import (
    get_data_options,
    get_subsetting_options,
    get_data,
)

import warnings
warnings.filterwarnings("ignore")

# import functions needed for testing
from climakitae.core.constants import UNSET
from climakitae.core.paths import VARIABLE_DESCRIPTIONS_CSV_PATH
from climakitae.explore.typical_meteorological_year import is_HadISD
from climakitae.explore.standard_year_profile import (
    _get_clean_standardyr_filename,
    _check_stations,
    _check_cached_area,
    _check_lat_lon,
    match_str_to_wl,
)
from climakitae.util.utils import read_csv_file
from climakitae.util.warming_levels import get_gwl_at_year

In [2]:
variable = "Air Temperature at 2m"
units = "degF"
qtile = 0.5
warming_levels = [1.5]
no_delta = True

# warming level window size
valid_window = 5
invalid_window_1 = 2
invalid_window_2 = 5.5

# station name options
station_list = [
    "Sacramento Executive Airport (KSAC)",
    "Santa Barbara Municipal Airport (KSBA)",
]
station_name = ["Sacramento Executive Airport (KSAC)"]
custom_name = ["Custom Station Name"]
mixed_list = [
    "Custom Station Name",
    "Santa Barbara Municipal Airport (KSBA)",
]
custom_list = [
    "Custom Name 1",
    "Custom Name 2",
]

# lat lon
# latitude = 34.4041
# longitude = -121.5160 # no data found for these lat and lon bounds!
latitude = 34.19966
longitude = -118.36543  # no data found for these lat and lon bounds!

# cached area
area_name = "Los Angeles County"

### Time-based file modification

Valid inputs to test
- "approach" = "Time" with a "centered_year" between 2015 and 20199
- no "warming_level" input

Invalid inputs to test
1. "approach" = "Time" with a "centered_year" between 2015 and 2099, with "warming_level" input
2. "approach" = "Time" and no "centered_year" provided
3. "centered_year" outside of 2015-2099
4. "approach" that is not "Time" or "Warming Level"

In [3]:
profile_selections = {
    "variable": variable,
    "resolution": "3 km",
    "q": qtile,
    #"warming_level": [1.5],
    "units": units,
    "no_delta": no_delta,
    # approach
    "approach": "Time",
    "centered_year": 2016,
    "time_profile_scenario": "SSP 3-7.0",
    # warming level window
    "warming_level_window": 5,
    # Location options -- uncomment based on your desired location type
    # "stations": station_name,  # uncomment for a weather station
    "latitude": (
        latitude - 0.02,
        latitude + 0.02,
    ),  # uncomment for a using a custom coordinate location
    "longitude": (
        longitude - 0.02,
        longitude + 0.02,
    ),  # uncomment for a custom coordinate location
    # "cached_area": area_name, # uncomment for a cached area
}

In [ ]:
profile = get_climate_profile(**profile_selections)

In [ ]:
export_profile_to_csv(profile, **profile_selections)

In [ ]:
get_gwl_at_year(1950, "SSP 3-7.0")

In [ ]:
downscaling_method = "Dynamical"
resolution = "3 km"
timescale = "hourly"
tidy = True
enable_hidden_vars = False

In [ ]:
# try out:
# SSP 2-4.5 ->
# SSP 5-8.5 ->
# SSP 3-7.0 ->

get_data_options(
    variable = None, downscaling_method = downscaling_method, resolution = None, timescale = timescale, scenario="SSP 3-7.0"
)

How...I am so confused. Do SSPs correspond to certain models?

In [4]:
retrieve_params = {
    "variable": variable,
    "resolution": "3 km",
    # "q": qtile,
    #"warming_level": [1.13],
    "units": units,
    # "no_delta": False,
    # # approach
    "approach": "Time",
    "centered_year": 2016,
    "time_profile_scenario": "SSP 3-7.0",
    # # warming level window
    "warming_level_window": 5,
    # # Location options -- uncomment based on your desired location type
    # "stations": station_name,  # uncomment for a weather station
    # "latitude": (
    #     latitude - 0.02,
    #     latitude + 0.02,
    # ),  # uncomment for a using a custom coordinate location
    # "longitude": (
    #     longitude - 0.02,
    #     longitude + 0.02,
    # ),  # uncomment for a custom coordinate location
    "cached_area": area_name,  # uncomment for a cached area
}

In [ ]:
# try out:
# SSP 2-4.5 ->
# SSP 5-8.5 ->
# SSP 3-7.0 ->

get_gwl_at_year(2016, "SSP 5-8.5")

In [ ]:
historic_data, future_data = retrieve_profile_data(**retrieve_params)

You have chosen to produce a time-based Standard Year climate profile centered around 2016 
and using scenario SSP 3-7.0 
Standard year functionality for time-based profiles identifies the closest warming level at that centered year 
for either the input SSP scenario or default 'SSP 3-7.0' if no scenario input is provided.The corresponding global warming level for input centered year 2016 will now 
be determined and used to produce the profile.
Corresponding warming level for 'centered_year'=2016 is [1.12]. 
Now producing the Standard Year climate profile at this warming level.


In [ ]:
future_data['simulation']

In [ ]:
# try out:
# SSP 2-4.5 ->
# SSP 5-8.5 ->
# SSP 3-7.0 ->

centered_year = 2016
scenario = "SSP 2-4.5"

In [ ]:
gwl_options = get_gwl_at_year(centered_year, scenario)
[float(gwl_options.loc[scenario, "Mean"])]

#### Filter by SSP

try out:

SSP 2-4.5 ->

SSP 5-8.5 ->

SSP 3-7.0 ->

In [ ]:
historic_data, future_data = retrieve_profile_data(**retrieve_params)

In [ ]:
historic_data["simulation"]

In [ ]:
test_historic = _filter_by_ssp(historic_data, "SSP 5-8.5")

In [ ]:
test_historic['simulation']